In [80]:
import time
import re
from bs4 import BeautifulSoup #use beautifulsoup to scrape data
from selenium import webdriver


url = "http://millercenter.org/the-presidency/presidential-speeches"

browser = webdriver.Chrome() #change this line depending on your system
browser.get(url) #go to the website 


SCROLL_PAUSE_TIME = 2




# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(browser.page_source, 'lxml')

In [81]:
import urllib3 
import pandas as pd
from datetime import datetime
import time
import string
import numpy as np



all_hyperlinks = []

for link in soup.findAll('a', href=True):
    if link != None:
        all_hyperlinks.append(link['href'])

# now I want the link strings containting /the-presidency/ within the list all_hyperlinks
speech_links = []

for element in all_hyperlinks:
    if element.find("/presidential-speeches/") >= 0: 
        speech_links.append(element)
        


http = urllib3.PoolManager()
raw_speeches=[]
president_names=[]
for element in speech_links:
    url = "http://millercenter.org" + element
    response = http.request('GET', url)
    soup_2 = BeautifulSoup(response.data, 'lxml')
    raw_speeches.append(soup_2.findAll('div', {"class":"transcript-inner"}))
    president_names.append(soup_2.find('p', {"class":"president-name"}).get_text(strip=True))
    


#converting into string due to weird beautifulsoup output 

str_raw_speeches
for i in range(len(raw_speeches)):
    if raw_speeches[i]:
        str_raw_speeches.append(str(raw_speeches[i]))
    else:
        
        #This extra step was done to deal with an inconsistency in the website. More information will be included in the readme
        
        url = "http://millercenter.org" + speech_links[i]
        response = http.request('GET', url)
        soup_2 = BeautifulSoup(response.data, 'lxml')
        raw_speeches.append(str(soup_2.findAll('div', {"class":"view-transcript"})))


#This will be used to clean the text from html tags
def tag_cleanr(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

cleaned_speeches = []

#These replace lines could and should be replaced with regex since we covered that in the syllabus more extensively but this works

for i in range(len(str_raw_speeches)):
    x = tag_cleanr(str_raw_speeches[i])
    x = x.replace('Transcript', '').replace('\r\n', '').replace('\n', '')
    cleaned_speeches.append(x)


#Get all the speech dates into a list

date_pattern = r'(\w+-\d+-\d{4})'
speech_dates=[]


for i in range(len(speech_links)):
    match = re.findall(date_pattern, speech_links[i])
    speech_dates.append(str(match))


for i in range(len(speech_dates)):
    #this feels like its terrible syntax but not sure if there is a nicer way, maybe I will replace it with regex
    
    speech_dates[i] = speech_dates[i].replace('[', '').replace(']', ''.replace("'", "")).replace("'", "")
    speech_dates[i] = datetime.strptime(speech_dates[i], '%B-%d-%Y')


df = pd.DataFrame({'dates':speech_dates, 'speeches':cleaned_speeches, 'president':president_names})

#this part is added because dataframe does not recognize [] as an empty input. 

df['speeches'].replace('[]', np.nan, inplace=True)

#now that I replaced [] with NaN, I can simply say drop all the rows with NaN elements

df.dropna(subset=['speeches'], inplace=True)

df = df.set_index('dates')
df = df.truncate(after=pd.Timestamp('1901-09-14'))


In [82]:
df.to_pickle("./speech_data.pkl")

In [88]:
df

,speeches,president
dates,,
2018-03-19,"[THE PRESIDENT: Thank you to our First Lady, M...",Donald Trump
2018-02-23,[THE PRESIDENT: Thank you very much. Thank you...,Donald Trump
2018-02-15,"[THE PRESIDENT: My fellow Americans, today I s...",Donald Trump
2018-02-01,"[THE PRESIDENT: Thank you, Paul and Mitch, for...",Donald Trump
2018-01-30,"[Mr. Speaker, Mr. Vice President, Members of C...",Donald Trump
2018-01-26,"[PRESIDENT TRUMP: Thank you, Klaus, very much....",Donald Trump
2017-12-18,[THE PRESIDENT: Thank you very much. Thank you...,Donald Trump
2017-09-19,"[PRESIDENT TRUMP: Mr. Secretary General, Mr. P...",Donald Trump
2017-07-24,"[TRUMP: Thank you, everybody. Thank you very m...",Donald Trump
